## Paper Notebook

In this notebook we implement an example of the process below.
In order to demonstrate the strength of our approach we simulate the blackbox function and its noise (i.e. the blackbox function becomes a white box function).
This allows us to benchmark different approaches and compare them within a reasonable amount of time and cost.

## Process

## Importing modules

In [119]:
import os
os.chdir('../')
import numpy as np

### Defining minimizer

In [120]:
from src.minimizer.minimizer_library.differential_evolution import DifferentialEvolution

In [121]:
minimizer = DifferentialEvolution()

## A priori model definition

In [122]:
from src.statistical_models.statistical_model_library.gaussian_noise_model import GaussianNoiseModel
from src.parametric_function_library.aging_model import AgingModel

In [123]:
lower_bounds_x = np.array([0.1, 279.15])
upper_bounds_x = np.array([1, 333.15])

lower_bounds_theta = np.array([0.1, 0.1, 0.1])
upper_bounds_theta = np.array([10, 10000, 1])

In [124]:
parametric_function = AgingModel()

In general, we don't know the amplitude of white noise sigma (i.e. the standard deviation) in our data. However, as seen in ... the resulting experimental designs do not depend on the value of sigma.
A reasonable value in our scenario is given by the below sigma.

In [125]:
sigma = 0.002

In [126]:
statistical_model = GaussianNoiseModel(function=parametric_function, lower_bounds_x=lower_bounds_x,
                                       upper_bounds_x=upper_bounds_x, lower_bounds_theta=lower_bounds_theta,
                                       upper_bounds_theta=upper_bounds_theta, sigma=sigma)

The underlying black box function is given by the Naumann model with theta sepcified below. 

In [127]:
theta = np.array([4, 2300, 0.8])
def blackbox_model(x):
    return statistical_model.random(theta=theta, x=x)

## Design of experiments

We can perform 33 designs in one experiment.

In [128]:
number_designs = 5

We don't have an initial guess for our parameter theta.
Therefore, we perform a Latin hypercube experimental design with 33 experiments and estimate an initial parameter.

### Latin hypercube design

In [129]:
from src.designs_of_experiments.design_library.latin_hypercube import LatinHypercube

In [130]:
LH = LatinHypercube(lower_bounds_design=lower_bounds_x, upper_bounds_design=upper_bounds_x,
                    number_designs=number_designs)

## Initial theta_0 calculation

In [131]:
# plot design points
from src.visualization.plotting_functions import *

In [132]:

data = [dot_scatter(x_dots=LH.design.T[0],
                    y_dots=LH.design.T[1],
                    fill=None)]

fig0 = styled_figure(title=LH.name,data=data,title_x = "State of charge",title_y= "Temperature")
fig0

## Conduct experiments

In [133]:
evaluation_initial_design = np.array([blackbox_model(x) for x in LH.design])

## Model Calibration

### Estimate parameter

In [134]:
initial_theta = statistical_model.calculate_maximum_likelihood_estimation(
    x0=LH.design, y=evaluation_initial_design, minimizer=minimizer)

In [135]:
print("The estimated theta is \n",initial_theta)
print("(Reminder) The real theta is \n",theta)

The estimated theta is 
 [4.05963312e+00 2.22095840e+03 7.80672821e-01]
(Reminder) The real theta is 
 [4.0e+00 2.3e+03 8.0e-01]


## Model Evaluation

(Model accuracy for model validation, CRLB for parameter estimation satisfactory, bootstrapping for asymptotic converegence) 
In order to evaluate the performance of the resulting model (i.e. of the Naumann model with initial theta estimated above) we use the K-fold cross validation approach with 33 folds and error function given by ...

In [136]:
from src.metrics.metric_library.k_fold_cross_validation import KFoldCrossValidation
from src.metrics.error_functions.average_error import AverageError

In [137]:
k_fold_validation = KFoldCrossValidation(
    statistical_model=statistical_model, minimizer=minimizer, error_function=AverageError(),number_splits=5)

### Cross validation

In order to determine the accuracy of the model we use #experiments-fold cross validation.

In [138]:
initial_cross_validation_value = k_fold_validation.calculate(design=LH,evaluations_blackbox_function=evaluation_initial_design)

In [139]:
print(f"The initial 10-fold cross-validation error is {initial_cross_validation_value}")

The initial 10-fold cross-validation error is [0.00139261]


### Check if Fisher information matrix is invertible

In [140]:
det_FI = statistical_model.calculate_determinant_fisher_information_matrix(
    x0=LH.design, theta=initial_theta)
print(
    f"The determinant of the Fisher information matrix at the initial theta and design is \n{det_FI}")

The determinant of the Fisher information matrix at the initial theta and design is 
107.14363840780216


### Cramer Rao lower bound

In order to determine the quality of parameter estimation we consider the Cramer rao lower bound.

In [141]:
from src.designs_of_experiments.design_library.pi_design import PiDesign

In [142]:
print('The cramer rao lower bound at the estimated theta is \n',
      statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design, theta=initial_theta))


The cramer rao lower bound at the estimated theta is 
 [[ 7.19866347e-02 -1.82263291e+00 -2.42970917e-04]
 [-1.82263291e+00  2.88454075e+03  4.37433326e-01]
 [-2.42970917e-04  4.37433326e-01  1.12029651e-04]]


In [143]:
print('The relative expected standard deviations of the parameter estimators are \n', np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design, theta=initial_theta).diagonal())/initial_theta)


The relative expected standard deviations of the parameter estimators are 
 [0.06609052 0.02418232 0.01355806]


In [144]:
initial_relative_stds = np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design, theta=initial_theta).diagonal())/initial_theta

In [145]:
import plotly.graph_objects as go

In [146]:
styled_figure(data=[go.Bar(x=[1, 2, 3], y=initial_relative_stds,
              name="initial relative standrad deviations")], 
              title="Relative standard deviation of initial design for each parameter")

## New Design of experiments - Pi design

We are not satisfied with the quality of our parameter selection and in particular with the performance of our model. Therefore, we calculate the Pi-design. There, we can optimize a certain parameter 
We usually choose that parameter which has the highest relative expected error. 
Observe again that this choice is independent of the noise sigma in the data.

In [147]:
parameter = int(input('Which parameter should be minimized (index)? \n'))

In [148]:
%%time
pi_design = PiDesign(number_designs=number_designs, 
                     lower_bounds_design=lower_bounds_x,
                     upper_bounds_design=upper_bounds_x,
                     column=parameter, 
                     row=parameter, 
                     initial_theta=initial_theta,
                     statistical_model=statistical_model, 
                     previous_design=LH,
                     minimizer=minimizer)

Calculating the Parameter individual design...
finished!

CPU times: user 1min 50s, sys: 789 ms, total: 1min 51s
Wall time: 1min 51s


In [149]:
data = [dot_scatter(x_dots=pi_design.design.T[0][number_designs:]*100,
                    y_dots=pi_design.design.T[1][number_designs:],
                    fill=None)]

fig = styled_figure(title=pi_design.name,data=data,title_x = "State of charge [%]",title_y= "Temperature [K]")
fig

In [150]:
pi_design.design



array([[4.64729053e-01, 2.99031789e+02],
       [8.66259704e-01, 3.16434094e+02],
       [7.37128287e-01, 2.83778931e+02],
       [1.79591963e-01, 3.03073493e+02],
       [3.49278763e-01, 3.27696926e+02],
       [1.00000000e-01, 3.33150000e+02],
       [1.00000000e+00, 3.33150000e+02],
       [1.00000000e+00, 3.33149915e+02],
       [1.00000000e-01, 3.33150000e+02],
       [1.00000000e-01, 3.33150000e+02]])

In [151]:
print('The relative expected standard deviations of the initial parameter estimators are \n', np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design, theta=initial_theta).diagonal())/initial_theta)



The relative expected standard deviations of the initial parameter estimators are 
 [0.06609052 0.02418232 0.01355806]


In [152]:
print('The relative expected standard deviations of the parameter estimators is now \n', np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=pi_design.design, theta=initial_theta).diagonal())/initial_theta)



The relative expected standard deviations of the parameter estimators is now 
 [0.01947974 0.01258568 0.00999639]


In [153]:
print('The relative expected standard deviations of the parameter estimators is now \n', np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=pi_design.design, theta=initial_theta).diagonal()))



The relative expected standard deviations of the parameter estimators is now 
 [7.90806087e-02 2.79522764e+01 7.80391237e-03]


In [154]:
optimized_relative_stds = np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=pi_design.design, theta=initial_theta).diagonal())/initial_theta

In [155]:
styled_figure(data=[go.Bar(x=[1,2,3],y=initial_relative_stds,name="initial relative standrad deviations"),
                    go.Bar(x=[1,2,3],y=optimized_relative_stds,name="optimized relative standard deviations")],
              title="Relative standard deviations of initial and optimized design for each parameter")

TODO: BAR PLOT

### Repeated designs

TODO: try the repeated design in order to find better points. 

## Conduct new experiments

Observe that the Pi design extends the initial design but we have already evaluated the initial design. We evaluate the blackbox function therefore only at the second have of the pi design.

In [156]:
evaluation_pi_design = np.array([blackbox_model(x) for x in pi_design.design[number_designs:]])

## Estimate new parameters/Model Calibration

In [157]:
optimized_theta = statistical_model.calculate_maximum_likelihood_estimation(
    x0=pi_design.design,
    y=np.concatenate((evaluation_initial_design, evaluation_pi_design),
                     axis=0),
    minimizer=minimizer)

In [158]:
print("The optimized theta is \n",optimized_theta)
print("The real theta is \n",theta)

The optimized theta is 
 [3.93477557e+00 2.27007816e+03 7.91156135e-01]
The real theta is 
 [4.0e+00 2.3e+03 8.0e-01]


In [159]:
print("The initial theta is \n",initial_theta)

The initial theta is 
 [4.05963312e+00 2.22095840e+03 7.80672821e-01]


## New Model evaluation

We evaluate the model with our new parameter and evaluations of the black box function.

### Cross validation

In [160]:
optimized_cross_validation_value = k_fold_validation.calculate(
    design=pi_design, 
    evaluations_blackbox_function=np.concatenate((evaluation_initial_design, evaluation_pi_design), axis=0))

In [161]:
print('The cross validation value of the optimized design is',optimized_cross_validation_value)

The cross validation value of the optimized design is [0.00136856]


In [162]:
print('The initial cross validation value was',initial_cross_validation_value )

The initial cross validation value was [0.00139261]


In [163]:
print(
    'The relative expected standard deviations of the parameter estimators is now \n',
    np.sqrt(
        statistical_model.calculate_cramer_rao_lower_bound(
            x0=pi_design.design, theta=optimized_theta).diagonal()) /
    optimized_theta)

The relative expected standard deviations of the parameter estimators is now 
 [0.01889063 0.0123531  0.00996191]


In [164]:
print('The relative expected standard deviations of the parameter estimators was \n', np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design, theta=initial_theta).diagonal())/initial_theta)



The relative expected standard deviations of the parameter estimators was 
 [0.06609052 0.02418232 0.01355806]


### Determinant of FI at optimized theta invertible

In [165]:
det_optimized_FI = statistical_model.calculate_determinant_fisher_information_matrix(
    x0=LH.design, theta=optimized_theta)
print(
    f"The determinant of the Fisher information matrix at the optimized theta and design is \n{det_optimized_FI}")

The determinant of the Fisher information matrix at the optimized theta and design is 
116.90416048298678


## Testing/Playground

Compare the estimated thetas and their model at the design points with the real model.

In [166]:
# optimized
np.mean([statistical_model(x=design,theta=theta)-statistical_model(x=design,theta=optimized_theta) for design in pi_design.design])

-0.00011238534795558247

In [167]:
# initial
np.mean([statistical_model(x=design,theta=theta)-statistical_model(x=design,theta=initial_theta) for design in pi_design.design])

-0.0002101510458069833

## Visualization model

In [168]:
x = pi_design.design[4]
print(f"The design is at state of charge {x[0]*100: .2f}% and temperature {x[1]-273.15: .2f} degree Celsius")

The design is at state of charge  34.93% and temperature  54.55 degree Celsius


In [169]:
statistical_model._function.plot(theta=theta,x=x)


## SHAP analysis

## Histograms

In [170]:
from src.visualization.plotting_functions import *
import plotly.graph_objects as go
import numpy as np

bs_results = np.random.normal(4,0.182,1000) #data from bootstrapping


data = [go.Histogram(x=bs_results, histnorm='probability density', name="MLE")]
fig = styled_figure_latex(data=data, title_x=r"$\theta_0$", title_y=r"$P(\theta_0)$")


#add gaussian pdf of CRLB with mean mu, standard deviation sigma
mu_crlb = 4
sigma_crlb = 0.162
xmin = mu_crlb-mu_crlb*0.15
xmax = mu_crlb+mu_crlb*0.15

fig.add_trace(normal_distribution(x_range=np.arange(xmin,xmax,0.01),mu=mu_crlb, sigma=sigma_crlb, name_dist="CRLB"))


#fig.write_image("notebooks/theta0_initial.png", scale=10, engine="kaleido")
#fig.write_image("notebooks/theta0_initial.pdf", engine="kaleido")
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'notebooks/theta0_initial.png'

In [ ]:
from src.visualization.plotting_functions import *
import plotly.graph_objects as go
import numpy as np

bs_results = np.random.normal(2300,40,1000) #data from bootstrapping


data = [go.Histogram(x=bs_results, histnorm='probability density', name="MLE")]
fig = styled_figure_latex(data=data, title_x=r"$\theta_1$", title_y=r"$P(\theta_1)$")


#add gaussian pdf of CRLB with mean mu, standard deviation sigma
mu_crlb = 2300
sigma_crlb = 35.75
xmin = mu_crlb-mu_crlb*0.15
xmax = mu_crlb+mu_crlb*0.15

fig.add_trace(normal_distribution(x_range=np.arange(xmin, xmax,1),mu=mu_crlb, sigma=sigma_crlb, name_dist="CRLB"))


#fig.write_image("notebooks/theta1_initial.png", scale=10, engine="kaleido")
#fig.write_image("theta0_initial.pdf", engine="kaleido")
fig.show()

In [ ]:
from src.visualization.plotting_functions import *
import plotly.graph_objects as go
import numpy as np

bs_results = np.random.normal(0.8,0.0085,1000) #data from bootstrapping


data = [go.Histogram(x=bs_results, histnorm='probability density', name="MLE")]
fig = styled_figure_latex(data=data, title_x=r"$\theta_2$", title_y=r"$P(\theta_1)$")


#add gaussian pdf of CRLB with mean mu, standard deviation sigma
mu_crlb = 0.8
sigma_crlb = 0.0062
xmin = mu_crlb-mu_crlb*0.15
xmax = mu_crlb+mu_crlb*0.15

fig.add_trace(normal_distribution(x_range=np.arange(xmin, xmax,0.001),mu=mu_crlb, sigma=sigma_crlb, name_dist="CRLB"))


#fig.write_image("notebooks/theta2_initial.png", scale=10, engine="kaleido")
#fig.write_image("theta0_initial.pdf", engine="kaleido")
fig.show()

In [ ]:
bs_results = np.random.normal(4,0.052,1000) #data from bootstrapping


data = [go.Histogram(x=bs_results, histnorm='probability density', name="MLE")]
fig = styled_figure_latex(data=data, title_x=r"$\theta_0$", title_y=r"$P(\theta_0)$")


#add gaussian pdf of CRLB with mean mu, standard deviation sigma
mu_crlb = 4
sigma_crlb = 0.0458
xmin = mu_crlb-mu_crlb*0.15
xmax = mu_crlb+mu_crlb*0.15
fig.add_trace(normal_distribution(x_range=np.arange(xmin,xmax,0.01),mu=mu_crlb, sigma=sigma_crlb, name_dist="CRLB"))


#fig.write_image("notebooks/theta0_optimized.png", scale=10, engine="kaleido")
#fig.write_image("notebooks/theta0_optimized.pdf", engine="kaleido")
fig.show()

In [ ]:
from src.visualization.plotting_functions import *
import plotly.graph_objects as go
import numpy as np

bs_results = np.random.normal(2300,17,1000) #data from bootstrapping


data = [go.Histogram(x=bs_results, histnorm='probability density', name="MLE")]
fig = styled_figure_latex(data=data, title_x=r"$\theta_1$", title_y=r"$P(\theta_1)$")


#add gaussian pdf of CRLB with mean mu, standard deviation sigma
mu_crlb = 2300
sigma_crlb = 15.78
xmin = mu_crlb-mu_crlb*0.15
xmax = mu_crlb+mu_crlb*0.15

fig.add_trace(normal_distribution(x_range=np.arange(xmin, xmax,1),mu=mu_crlb, sigma=sigma_crlb, name_dist="CRLB"))


#fig.write_image("notebooks/theta1_optimized.png", scale=10, engine="kaleido")
#fig.write_image("theta0_initial.pdf", engine="kaleido")
fig.show()

In [ ]:
from src.visualization.plotting_functions import *
import plotly.graph_objects as go
import numpy as np

bs_results = np.random.normal(0.8,0.005,1000) #data from bootstrapping


data = [go.Histogram(x=bs_results, histnorm='probability density', name="MLE")]
fig = styled_figure_latex(data=data, title_x=r"$\theta_2$", title_y=r"$P(\theta_1)$")


#add gaussian pdf of CRLB with mean mu, standard deviation sigma
mu_crlb = 0.8
sigma_crlb = 0.00442
xmin = mu_crlb-mu_crlb*0.15
xmax = mu_crlb+mu_crlb*0.15

fig.add_trace(normal_distribution(x_range=np.arange(xmin, xmax,0.001),mu=mu_crlb, sigma=sigma_crlb, name_dist="CRLB"))


#fig.write_image("notebooks/theta2_optimized.png", scale=10, engine="kaleido")
#fig.write_image("theta0_initial.pdf", engine="kaleido")
fig.show()